In [1]:
import logging
import sys

from omegaconf import OmegaConf

from adhoc.bm25_retriever import build_bm25_scoring_fn
from adhoc.eval_helper.retreival_exp_helper import run_retrieval_eval_report_w_conf
from adhoc.other.bm25_retriever_helper import get_tokenize_fn
from adhoc.other.bm25t_retriever import BM25T_Retriever2
from adhoc.test_code.inv_index_test import InvIndexReaderClient
from dataset_specific.msmarco.passage.doc_indexing.retriever import load_bm25_resources, get_bm25_stats_from_conf
from trainer_v2.chair_logging import c_log
from trainer_v2.per_project.transparency.mmp.bm25t_helper import load_align_scores

INFO	chair 	11-27 14:47:33 Chair logging init


In [2]:
import os

os.chdir("/mnt/nfs/work3/youngwookim/code/Chair/")

In [3]:

c_log.setLevel(logging.INFO)
conf_path = "confs/experiment_confs/bm25t/bm25t_table3_dev100_retrieval.yaml"
conf = OmegaConf.load(conf_path)

table = load_align_scores(conf.table_path)
bm25_conf = OmegaConf.load(conf.bm25conf_path)
avdl, cdf, df, dl, inv_index = load_bm25_resources(bm25_conf)
scoring_fn = build_bm25_scoring_fn(cdf, avdl)
tokenize_fn = get_tokenize_fn(bm25_conf)




INFO	chair 	11-27 14:47:36 145929 entry loaded
INFO	chair 	11-27 14:47:36 Loading document frequency (df)
INFO	chair 	11-27 14:47:37 Loading document length (dl)
INFO	chair 	11-27 14:47:42 Loading inv_index
INFO	chair 	11-27 14:50:49 Done


In [83]:
from dataset_specific.msmarco.passage.doc_indexing.retriever import load_bm25_resources, get_bm25_stats_from_conf
get_bm25_stats_from_conf
avdl, cdf, df, dl_d = get_bm25_stats_from_conf(bm25_conf, None)


INFO	chair 	11-27 19:15:56 Loading document frequency (df)
INFO	chair 	11-27 19:15:57 Loading document length (dl)
INFO	chair 	11-27 19:16:01 Done


In [95]:
from collections import Counter, defaultdict
from typing import List, Iterable, Callable, Dict, Tuple, Set
from adhoc.bm25 import BM25_verbose

from adhoc.bm25_retriever import RetrieverIF
from adhoc.kn_tokenizer import KrovetzNLTKTokenizer
from list_lib import left
from misc_lib import get_second, dict_to_tuple_list
from typing import List, Iterable, Dict, Tuple, Set

from trainer_v2.chair_logging import c_log


In [4]:
posting_throb = inv_index["throb"]
posting_scratchy = inv_index["scratchy"]

In [5]:
print(len(posting_water))
len(posting_question_mark)

NameError: name 'posting_water' is not defined

In [6]:
import pickle
pickle.dump(posting_throb, open("/mnt/nfs/work3/youngwookim/code/Chair/data/cache/posting_throb.pickle", "wb"))
pickle.dump(posting_scratchy, open("/mnt/nfs/work3/youngwookim/code/Chair/data/cache/posting_scratchy.pickle", "wb"))


In [11]:
import random
def sample_inv_index(inv_index):
    compress_rate = 0.1
    new_inv_index = {}
    for q_term, entries in inv_index.items():
        n_sample = int(len(entries) * compress_rate)
        random.sample(entries, n_sample)
        new_inv_index[q_term] = random.sample(entries, n_sample)

    return new_inv_index




In [12]:
sampled_inv_index = sample_inv_index(inv_index)


In [109]:
from collections import Counter, defaultdict
from typing import List, Iterable, Callable, Dict, Tuple, Set, TypeVar

from adhoc.bm25_retriever import RetrieverIF
from adhoc.kn_tokenizer import KrovetzNLTKTokenizer
from list_lib import left
from misc_lib import get_second, dict_to_tuple_list
from typing import List, Iterable, Dict, Tuple, Set

from trainer_v2.chair_logging import c_log
DocID = TypeVar("DocID")


class BM25_Retriever2(RetrieverIF):
    def __init__(
            self,
            get_posting: Callable[[str], List[Tuple[DocID, int]]],
            df,
            dl_d,
            scoring_fn,
            tokenize_fn,
    ):
        self.get_posting = get_posting
        self.scoring_fn = scoring_fn
        self.df = df
        self.tokenize_fn = tokenize_fn
        self.dl_d = dl_d

    def retrieve(self, query, n_retrieve=1000) -> List[Tuple[DocID, float]]:
        c_log.debug("Query: %s", query)
        q_tokens = self.tokenize_fn(query)
        q_tf = Counter(q_tokens)
        q_terms = list(q_tf.keys())
        q_term_df_pairs = []
        for t in q_terms:
            df = self.df[t]
            q_term_df_pairs.append((t, df))

        # Search rare query term first
        q_term_df_pairs.sort(key=get_second)
        b = 1.4
        k1 = 0.1
        k2 = 100
        doc_score: Dict[DocID, float] = Counter()
        doc_info = defaultdict(list)
        for idx, (q_term, qterm_df) in enumerate(q_term_df_pairs):
            qf = q_tf[q_term]
            max_tf: Dict[str, float] = {}

            target_term_postings = self.get_posting(q_term)
            c_log.debug("Update counts")
            for doc_id, cnt in target_term_postings:
                max_tf[doc_id] = cnt

            for doc_id, cnt in max_tf.items():
                tf = cnt
                dl = self.dl_d[doc_id]
                t = BM25_verbose(tf, qf, qterm_df, cdf, dl, avdl, b, k1, k2)
                t = self.scoring_fn(tf, qf, dl, qterm_df)
                doc_score[doc_id] += t
                doc_info[doc_id].append((q_term, t, tf, qf, qterm_df))
            c_log.debug("Done score updates")

        doc_score_pair_list = list(doc_score.items())
        doc_score_pair_list.sort(key=get_second, reverse=True)
        return doc_score_pair_list, doc_info


In [110]:
table = {}
retriever = BM25_Retriever2(get_posting, df, dl_d, scoring_fn, tokenize_fn)

In [111]:
ret, doc_info = retriever.retrieve("foods supplements lower blood sugar")
doc_score_d = dict(ret)

In [60]:
from table_lib import tsv_iter
from adhoc.bm25_class import BM25Bare
itr = tsv_iter("data/msmarco/dev_sample100/corpus.tsv")

In [61]:
corpus = list(itr)
target_qd = [e for e in corpus if e[0] == "188714"]

In [66]:
bm25_bare = BM25Bare(df, len(dl), avdl)

In [113]:
def analyze_for_doc(doc_id):
    target_doc = [e for e in target_qd if e[1] == doc_id]
    _,_, q, d = target_doc[0]
    tf = Counter(tokenize_fn(d))
    q_tfs = Counter(tokenize_fn(q))
    cur_dl = sum(tf.values())
    score_sum = 0
    info_a = []
    for q_term, qtf in q_tfs.items():
        t = bm25_bare.per_term_score(q_term, qtf, tf[q_term], cur_dl)
        score_sum += t
        info_a.append((q_term, t))

    score_a = score_sum
    score_b = doc_score_d[doc_id]
    info_b = doc_info[doc_id]
    print("<A>")
    print(score_a)
    print(info_a)
    print("<B>")
    print(score_b)
    print(info_b)
    

In [115]:
rel_doc_id = "8003843"
top_doc_id, top_doc_score = ret[0]

analyze_for_doc(rel_doc_id)
print(top_doc_id)
analyze_for_doc(top_doc_id)


<A>
21.75363010778936
[('foods', 4.633066883938325), ('and', 2.191803711245579), ('supplements', 0.0), ('to', 2.191803711245579), ('lower', 4.18212487091244), ('blood', 3.6688066841872735), ('sugar', 4.886024246260162)]
<B>
17.195213817629643
[('sugar', 4.968249632650397, 2, 1, 67911), ('foods', 4.421365108402322, 1, 1, 84818), ('lower', 3.9910282856775563, 1, 1, 133324), ('blood', 3.814570790899366, 3, 1, 230223)]
3573129


IndexError: list index out of range

In [107]:
target_qd

[['188714',
  '4321745',
  'foods and supplements to lower blood sugar',
  'Food And Supplements That Lower Blood Sugar Levels. Cinnamon: Researchers are finding that cinnamon reduces blood sugar levels naturally when taken daily. If you absolutely love cinnamon you can sprinkle the recommended six grams of cinnamon on your food throughout the day to achieve the desired effect.'],
 ['188714',
  '8003843',
  'foods and supplements to lower blood sugar',
  'Cinnamon a powerful anti-inflammatory and antioxidant, and helps lower blood sugar levels and improve insulin’s ability to respond to glucose in the blood. Sprinkle it on oatmeal, have it in tea form, sprinkle it on fruit! Prickly Pear/ Nopal: is one of the lowest foods on the glycemic index. Several studies in Mexico have shown blood sugar lowering effects. Grapefruit: Not only do grapefruits have plenty of vitamin C, they also contain naringenin, an antioxidant that appears to cut the risk of developing insulin resistance.'],
 ['188

In [116]:
len(ret)

485180